Microsoft Excel is a widely used spreadsheet application that has been an essential tool for businesses, financial institutions, and individuals for decades. However, manually updating and manipulating data in Excel can be time-consuming and error-prone. Fortunately, Python provides a way to automate many of these tasks, making data analysis and manipulation faster and more efficient. We will explore how to automate Excel spreadsheets using Python.
The most commonly used library for this purpose is openpyxl; a Python library that is used to read from an Excel file or write to an Excel file. Data scientists use Openpyxl for data analysis, data copying, data mining, drawing charts, styling sheets, adding formulas, and more.


We have been provided with the following files:
1. **Student_names.txt:**
|Student Name | Student Id |
2. **Student_results.csv:**
    |Student Id| Assignment 1| Assignment 2| Assignment 3| Midterm Exam| Final Exam|

Task 1: Combine the text file with the csv on the Student_Id into an excel workbook. The sheet name should be "Student Marks".
Task 2: Calculate the sum of marks each student obtained in Assignment 1, Assignment 2, Assignment 3, Midterm Exam and Final Exam.
Task 3: Grade each student based on the sum calculated in task 2 using the grade system below:
    percentage >= 75 - 100 : A
    percentage >= 70 - 74 : B
    percentage >= 50 - 69 : C
    percentage >= 45 - 49 : D
    percentage >= 40 - 45 : E
    percentage <= 39:  'F'


In [ ]:
#Install openpyxl
!pip install openpyxl

In [1]:
#Import libraries
import pandas as pd
from openpyxl import load_workbook

# Load the Excel file
results_file = pd.read_excel('student_results.xlsx')


In [ ]:
# Load the text file containing Course names and their IDs
name_file = pd.read_csv('student_names.csv', sep=",", header=None, names=['Student Name', 'Student ID'])

# Merge the Course name with the original Excel sheet
merged_file = pd.merge(results_file, name_file, on='Student ID', how='right')

# Save the merged Excel sheet with the new column
writer = pd.ExcelWriter('final_results.xlsx', engine='openpyxl')
merged_file.to_excel(writer, sheet_name='Student Marks', index=False)
writer.save()


In [3]:
name_file

,Student Name,Student ID
0,Kingsley Roderick,1682
1,Foswell Frederic,2180
2,Pym Henry,2323
3,Batroc Georges,2586
4,Barton Clinton Francis,2665
5,Jones Angelica,3923
6,Astrovic Vance,3956
7,Creed Victor,3985
8,Cassidy Sean,4362
9,Wilson Wade,4438


In [4]:
# Create a pivot table to calculate the total marks for each student in each course
pivot_table = pd.pivot_table(merged_file, values=['Assignment 1', 'Assignment 2', 'Assignment 3', 'Midterm Exam', 'Final Exam'], index=['Student ID','Student Name'], aggfunc='sum')

# Sum the marks for all courses for each student
pivot_table['Total marks'] = pivot_table.sum(axis=1)

#Grade the total marks
# Define a function to assign grades based on the percentage
def assign_grade(percentage):
    if percentage >= 75:
        return 'A'
    elif percentage >= 70:
        return 'B'
    elif percentage >= 50:
        return 'C'
    elif percentage >= 45:
        return 'D'
    elif percentage >= 40:
        return 'E'
    else:
        return 'F'
pivot_table['Grade'] = pivot_table['Total marks'].apply(assign_grade)
# Save the pivot table to a new sheet in the output Excel file
with pd.ExcelWriter('final_results.xlsx', engine='openpyxl', mode='a') as writer:
    pivot_table.to_excel(writer, sheet_name='Student Grades', index=True)


In [ ]:
import matplotlib.pyplot as plt
import openpyxl

# Calculate the class average for total marks
class_average = pivot_table['Total marks'].mean()

# Count the number of students in each category
below_50 = pivot_table[pivot_table['Total marks'] < 50]['Total marks'].count()
between_50_90 = pivot_table[(pivot_table['Total marks'] >= 50) & (pivot_table['Total marks'] < 90)]['Total marks'].count()
above_90 = pivot_table[pivot_table['Total marks'] >= 90]['Total marks'].count()

# Create a new sheet and add the class average and counts
with pd.ExcelWriter('final_results.xlsx', engine='openpyxl', mode='a') as writer:
    sheet = writer.book.create_sheet('Marks Statistics')

    # Add the class average and counts to the sheet
    sheet['B1'] = 'Count'
    sheet['A2'] = 'Total Marks Average'
    sheet['B2'] = class_average
    sheet['A3'] = 'Below 50%'
    sheet['B3'] = below_50
    sheet['A4'] = '50% to 90%'
    sheet['B4'] = between_50_90
    sheet['A5'] = '90% and above'
    sheet['B5'] = above_90

    # Plot the counts in a pie chart and add it to the sheet
    labels = ['Below 50%', '50% to 90%', '90% and above']
    sizes = [below_50, between_50_90, above_90]
    colors = ['red', 'yellowgreen', 'green']
    explode = (0.1, 0, 0)
    plt.pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='%1.1f%%', startangle=140)
    plt.title('Distribution of Total Marks')
    plt.savefig('pie_chart.png')
    img = openpyxl.drawing.image.Image('pie_chart.png')
    sheet.add_image(img, 'D1')
    writer.save()
